In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from IPython.display import Image

In [3]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [4]:
from we import get_we, initiate_model

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from scipy.stats import f_oneway

In [7]:
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [10]:
labels = [m['label'] for m in models]

In [11]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_nouns_we.csv'
                    )

In [12]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,762,763,764,765,766,767,Number,Gender,Lemma,Semantic
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.836139,0.551025,0.347197,0.789782,0.393246,0.300031,0.609232,0.913722,0.178287,0.838988,...,0.586894,0.674134,0.276217,0.081619,0.154076,0.553278,invariable,feminine,2D,Other
3D,0.828946,0.549880,0.339958,0.797102,0.395346,0.301288,0.603051,0.916539,0.185836,0.844714,...,0.584636,0.674752,0.278596,0.078074,0.143366,0.548315,invariable,feminine,3D,Other
a,0.826660,0.556389,0.332618,0.805891,0.399844,0.295803,0.597985,0.914675,0.201301,0.846672,...,0.577705,0.673679,0.278649,0.081689,0.139490,0.533415,invariable,masculine,a,Attribute
aa,0.826916,0.555891,0.332298,0.805754,0.400046,0.296797,0.597758,0.914183,0.201098,0.845985,...,0.578594,0.673429,0.278861,0.081218,0.139010,0.533594,invariable,masculine,aa,Other
abandon,0.826281,0.556306,0.332007,0.805256,0.399368,0.296458,0.597567,0.914983,0.200809,0.846577,...,0.578808,0.673362,0.279152,0.081560,0.139390,0.534123,singular,masculine,abandon,Other


In [13]:
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'Semantic'

# Nouns

In [14]:
pos = ['NOUN']

Split each model into train and test using k_fold cross validation:

In [20]:
X_noun_train = []
y_noun_train = []

X_noun_test = []
y_noun_test = []

In [67]:
X_noun_test[-1][0][y_noun_test[-1][0] == 1]

,0,1,2,3,4,5,6,7,8,9,...,756,757,758,759,760,761,762,763,764,765
Word,,,,,,,,,,,,,,,,,,,,,
juge,0.826999,0.556305,0.332741,0.806138,0.399868,0.295796,0.597677,0.914277,0.200987,0.846266,...,0.700008,0.512640,0.519824,0.997353,0.197873,0.371594,0.577533,0.673251,0.278180,0.081579
française,0.824371,0.555220,0.332107,0.799157,0.401698,0.301468,0.598216,0.918504,0.201774,0.845559,...,0.704230,0.520594,0.510857,0.995831,0.198841,0.376646,0.582651,0.671105,0.284217,0.079869
femme,0.826138,0.556252,0.332520,0.805565,0.399587,0.297251,0.597983,0.914046,0.201440,0.846156,...,0.701125,0.513597,0.518729,0.996630,0.198374,0.372877,0.578251,0.672666,0.278291,0.081134
citoyens,0.826304,0.555997,0.331438,0.805619,0.399750,0.297170,0.597740,0.913954,0.201278,0.845734,...,0.701905,0.512963,0.517433,0.996573,0.198081,0.374021,0.579020,0.673343,0.279074,0.081301
professeur,0.823892,0.554782,0.330126,0.798092,0.402014,0.300715,0.599464,0.918413,0.200676,0.845716,...,0.706020,0.522638,0.510830,0.995195,0.198739,0.376953,0.583459,0.670208,0.283817,0.080593
musiciens,0.824060,0.554802,0.330870,0.801526,0.399926,0.299850,0.597505,0.918802,0.201231,0.847934,...,0.706606,0.520884,0.511432,0.997359,0.200161,0.376278,0.581334,0.670382,0.282138,0.082070
candidat,0.824976,0.556532,0.331263,0.802573,0.400902,0.300917,0.597641,0.916143,0.203104,0.845831,...,0.703899,0.516425,0.511036,0.994916,0.198743,0.376588,0.583206,0.671899,0.283080,0.079744
conseillers,0.825939,0.556070,0.331554,0.804476,0.399656,0.297760,0.597027,0.915039,0.201134,0.845616,...,0.703151,0.513882,0.515892,0.996798,0.198384,0.376200,0.579279,0.672453,0.280372,0.080852
épouse,0.826211,0.556320,0.332777,0.805629,0.400047,0.296790,0.597891,0.914350,0.201516,0.846018,...,0.700312,0.513997,0.519464,0.997105,0.198126,0.372154,0.577675,0.673093,0.278377,0.081435


In [21]:
n_folds = 5

In [22]:
for we in we_with_features:
    X, y = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='Person',
                                           split=False,
                                           balance=True)
    X_trains = []
    y_trains = []
    
    X_tests = []
    y_tests = []
    
    X_folds = np.array_split(X, n_folds)
    y_folds = np.array_split(y, n_folds)
    
    for i in range(n_folds):
        X_trains.append(pd.DataFrame(np.concatenate(X_folds[:i] + X_folds[i+1:])))
        y_trains.append(np.concatenate(y_folds[:i] + y_folds[i+1:]))

        X_folds[i].columns = X_folds[i].columns.map(int)
        X_tests.append(X_folds[i])
        y_tests.append(y_folds[i])
        
    
    X_noun_train.append(X_trains)
    X_noun_test.append(X_tests)
    
    y_noun_train.append(y_trains)
    y_noun_test.append(y_tests)

In [23]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['1 fold size'] = [len(x[0]) for x in X_noun_test]
dataset_sizes

,1 fold size
flau_small_c,120
flau_base_u,124
flau_base_c,120
flau_large_c,120
cam_base,97
xlm_large,26
xlm_base,26
bert_base_u,56
distilbert_base,50
bert_base_c,50


## Non-independent dims

Calculate non-independendent dimensions for each of _k_ splits.

ANOVA test with p-value < 0.01:

In [24]:
anova_dims = []

In [25]:
for i in range(len(models)):
    anova_dims.append([])
    for j in range(n_folds):
        anova_dims[i].append(get_anova_dims(X_noun_train[i][j], y_noun_train[i][j]))

Get dimensions where MI > 0:

In [26]:
mi_dims = []

In [27]:
for i in range(len(models)):
    mi_dims.append([])
    for j in range(n_folds):
        mi_dims[i].append(get_mi_dims(X_noun_train[i][j], y_noun_train[i][j]))

In [28]:
for i in range(len(models)):
    for j in range(n_folds):
        temp = list(map(lambda x: int(x), mi_dims[i][j]))
        mi_dims[i][j] = temp

Get dimensions that are both found by the ANOVA independency test and MI test:

In [29]:
non_ind_dims = []

In [30]:
for i in range(len(models)):
    non_ind_dims.append([])
    for j in range(n_folds):
        non_ind_dims[i].append(list(set(anova_dims[i][j]).intersection(mi_dims[i][j])))

Stats about the number of dimensions for each model for the 1st split:

In [31]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [32]:
non_ind_df['ANOVA'] = [len(x[0]) for x in anova_dims]
non_ind_df['MI'] = [len(x[0]) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x[0]) for x in non_ind_dims]

In [33]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,208,346,173
flau_base_u,68,421,51
flau_base_c,265,491,207
flau_large_c,396,649,312
cam_base,12,459,10
xlm_large,4,443,2
xlm_base,1,451,1
bert_base_u,3,374,2
distilbert_base,54,413,36
bert_base_c,57,442,47


In [34]:
dims = {}

In [35]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = [X_noun_train[i][0].columns]*n_folds
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [36]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [37]:
lr_res = []

In [38]:
for i in range(len(models)):
    lr_res.append([])
    for j in range(n_folds):
        lr_res[i].append(lr(X_noun_train[i][j], y_noun_train[i][j]))

Train Perceptron:

In [39]:
perceptron_res = []

In [40]:
for i in range(len(models)):
    perceptron_res.append([])
    for j in range(n_folds):
        perceptron_res[i].append(perceptron(X_noun_train[i][j], y_noun_train[i][j]))

Compute correlation to the gender vector:

In [41]:
corr_res = []

In [42]:
for i in range(len(models)):
    corr_res.append([])
    for j in range(n_folds):
        corr_res[i].append(correlation(X_noun_train[i][j], y_noun_train[i][j]))

In [43]:
for i in range(len(models)):
    for alpha in alphas:
        dims[labels[i]][f'LR{alpha}'] = []
        dims[labels[i]][f'Perc{alpha}'] = []
        dims[labels[i]][f'Corr{alpha}'] = []
        dims[labels[i]][f'All imp dims{alpha}'] = []
        for j in range(n_folds):
            num_imp_dims = len(X_noun_test[i][0].columns)*alpha//100
            lr_dims = [x[0] for x in lr_res[i][j][:num_imp_dims]]
            perc_dims = [x[0] for x in perceptron_res[i][j][:num_imp_dims]]
            corr_dims = [x[0] for x in corr_res[i][j][:num_imp_dims]]
            dims[labels[i]][f'LR{alpha}'].append(lr_dims)
            dims[labels[i]][f'Perc{alpha}'].append(perc_dims)
            dims[labels[i]][f'Corr{alpha}'].append(corr_dims)
            dims[labels[i]][f'All imp dims{alpha}'].append(list(set(lr_dims).intersection(perc_dims).intersection(corr_dims)))

In [44]:
dims[labels[0]]['LR1']

[[121, 417, 0, 24, 358],
 [121, 295, 276, 70, 495],
 [121, 495, 295, 146, 375],
 [121, 295, 276, 495, 417],
 [121, 295, 375, 417, 364]]

## Compute medians

In [45]:
medians = {}

In [46]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else list(dims[labels[i]][dim_group])
            # Median of dimensions where feature vector is equal to 0
            median_0 = X_noun_train[i][j][y_noun_train[i][j] == 0][dim_list].median()
            # Median of dimensions where feature vector is equal to 1
            median_1 = X_noun_train[i][j][y_noun_train[i][j] == 1][dim_list].median()

            medians[labels[i]][dim_group].append({
                    '0': median_0,
                    '1': median_1
                })

We can compare the number of dimensions found by each test for each model in the first fold:

In [47]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group][0]) if dim_group != 'All dims' else \
                len(dims[model][dim_group])

In [48]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,5,5,5,5,5,5,5,5,5,5
ANOVA,208,68,265,396,12,4,1,3,54,57
MI,346,421,491,649,459,443,451,374,413,442
All non ind,173,51,207,312,10,2,1,2,36,47
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,1,0,0,0,0,2,1,0,1,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [49]:
y_preds = {}

In [50]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_preds[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else dims[labels[i]][dim_group]
            
            # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
            mae0 = X_noun_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['0'], x), axis=1)
            mae1 = X_noun_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['1'], x), axis=1)
            
            y_preds[labels[i]][dim_group].append((mae0 > mae1).apply(int))
    

## Compute accuracies

In [51]:
accs = {}

In [52]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        accs[labels[i]][dim_group] = []
        for j in range(n_folds):
            y_true = y_noun_test[i][j]
            y_pred = y_preds[labels[i]][dim_group][j]
            if any(y_pred):
                acc = accuracy_score(y_true, y_pred)
            else:
                acc = 0
            accs[labels[i]][dim_group].append(acc)

In [53]:
# Compute average accuracy between 5 folds
avg_accs = {}

In [54]:
for i in range(len(models)):
    avg_accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        avg_accs[labels[i]][dim_group] = np.average(accs[labels[i]][dim_group])

In [55]:
accs_df = pd.DataFrame(avg_accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.845000,0.607094,0.593333,0.850000,0.535309,0.399385,0.586769,0.416883,0.556980,0.553143
ANOVA,0.836667,0.629871,0.601667,0.843333,0.531186,0.413846,0.200000,0.485390,0.589388,0.638367
MI,0.840000,0.634710,0.595000,0.851667,0.535309,0.406462,0.579077,0.416883,0.552735,0.557224
All non ind,0.838333,0.629832,0.596667,0.845000,0.526997,0.444615,0.207692,0.456688,0.564980,0.638367
LR1,0.708333,0.573092,0.573333,0.690000,0.502384,0.499692,0.594769,0.449026,0.646122,0.540653
Perc1,0.681667,0.571309,0.603333,0.723333,0.558162,0.476615,0.570769,0.409675,0.577633,0.536653
Corr1,0.733333,0.611972,0.611667,0.758333,0.512479,0.437846,0.563385,0.449156,0.576653,0.626122
All imp dims1,0.130000,0.437398,0.230000,0.130000,0.102083,0.076923,0.115385,0.072727,0.108000,0.000000
LR5,0.803333,0.610372,0.635000,0.790000,0.529081,0.468923,0.540308,0.416818,0.621388,0.540980
Perc5,0.766667,0.615224,0.628333,0.805000,0.547680,0.446154,0.563385,0.416883,0.593959,0.536735


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [56]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,-0.008333,0.022777,0.008333,-0.006667,-0.004124,0.014462,-0.386769,0.068506,0.032408,0.085224
MI,-0.005000,0.027616,0.001667,0.001667,0.000000,0.007077,-0.007692,0.000000,-0.004245,0.004082
All non ind,-0.006667,0.022738,0.003333,-0.005000,-0.008312,0.045231,-0.379077,0.039805,0.008000,0.085224
LR1,-0.136667,-0.034002,-0.020000,-0.160000,-0.032925,0.100308,0.008000,0.032143,0.089143,-0.012490
Perc1,-0.163333,-0.035785,0.010000,-0.126667,0.022852,0.077231,-0.016000,-0.007208,0.020653,-0.016490
Corr1,-0.111667,0.004878,0.018333,-0.091667,-0.022831,0.038462,-0.023385,0.032273,0.019673,0.072980
All imp dims1,-0.715000,-0.169696,-0.363333,-0.720000,-0.433226,-0.322462,-0.471385,-0.344156,-0.448980,-0.553143
LR5,-0.041667,0.003278,0.041667,-0.060000,-0.006229,0.069538,-0.046462,-0.000065,0.064408,-0.012163
Perc5,-0.078333,0.008130,0.035000,-0.045000,0.012371,0.046769,-0.023385,0.000000,0.036980,-0.016408


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [57]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c LR50 0.015000000000000013
flau_base_u MI 0.027616050354051902
flau_base_c LR5 0.04166666666666663
flau_large_c LR25 0.010000000000000009
cam_base Perc1 0.022852233676975975
xlm_large LR1 0.10030769230769226
xlm_base LR1 0.008000000000000007
bert_base_u ANOVA 0.06850649350649352
distilbert_base LR1 0.08914285714285708
bert_base_c ANOVA 0.08522448979591823


In [58]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c LR50 0.86
flau_base_u MI 0.634710201940729
flau_base_c LR5 0.635
flau_large_c LR25 0.86
cam_base Perc1 0.5581615120274914
xlm_large LR1 0.49969230769230766
xlm_base LR1 0.5947692307692308
bert_base_u ANOVA 0.48538961038961037
distilbert_base LR1 0.6461224489795917
bert_base_c ANOVA 0.6383673469387754


For each fold find dimensions with the highest accuracy, and pick dimensions appeared in all 5 folds:

In [59]:
best_dims = {}

In [60]:
dim_cand_accs = {}

In [61]:
for i in range(len(labels)):
    bd = []
    for j in range(n_folds):
        best_dim_group = ''
        best_acc = 0
        for dim_group in accs[labels[i]].keys():
            if accs[labels[i]][dim_group][j] > best_acc:
                best_dim_group = dim_group
                best_acc = accs[labels[i]][dim_group][j]
        bd.extend(dims[labels[i]][best_dim_group][j])
    
    # Find dimensions that appeared within dimensions with the highest accuracy in all 5 folds
    d, c = np.unique(bd, return_counts=True)
    dim_cand = [x[0] for x in zip(d, c) if x[1] >= 5]
    best_dims[labels[i]] = dim_cand
    print(f'{labels[i]}: {len(dim_cand)} dimensions repeated in 5 folds')

    # For each fold build a prediction and calculate accuracy
    cand_accs = []
    for j in range(n_folds):
        med0_cand = X_noun_train[i][j][y_noun_train[i][j] == 0][dim_cand].median()
        med1_cand = X_noun_train[i][j][y_noun_train[i][j] == 1][dim_cand].median()
        
        mae0_cand = X_noun_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med0_cand, x), axis=1)
        mae1_cand = X_noun_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med1_cand, x), axis=1)
        
        y_pred_cand = (mae0_cand > mae1_cand).apply(int)
        if any(y_pred_cand):
            cand_accs.append(accuracy_score(y_true=y_noun_test[i][j], y_pred=y_pred_cand))
        else:
            cand_accs.append(0)
    dim_cand_accs[labels[i]] = cand_accs
    print(f'Average accuracy: {np.average(cand_accs)}')

flau_small_c: 95 dimensions repeated in 5 folds
Average accuracy: 0.8733333333333334
flau_base_u: 5 dimensions repeated in 5 folds
Average accuracy: 0.6184500393391031
flau_base_c: 1 dimensions repeated in 5 folds
Average accuracy: 0.6033333333333333
flau_large_c: 63 dimensions repeated in 5 folds
Average accuracy: 0.85
cam_base: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
xlm_large: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
xlm_base: 1 dimensions repeated in 5 folds
Average accuracy: 0.21538461538461537
bert_base_u: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
distilbert_base: 1 dimensions repeated in 5 folds
Average accuracy: 0.5891428571428572
bert_base_c: 6 dimensions repeated in 5 folds
Average accuracy: 0.6831020408163265


In [62]:
with open('../Data/best_results/semantic_person_dims.pickle', 'wb') as f:
    pickle.dump(dims, f)

In [63]:
with open('../Data/best_results/semantic_person_accs.pickle', 'wb') as f:
    pickle.dump(accs, f)

In [64]:
with open('../Data/best_results/semantic_person_medians.pickle', 'wb') as f:
    pickle.dump(medians, f)

# Conclusion